In [1]:
from langchain.agents import AgentType
from langchain.agents import initialize_agent
from langchain.llms import OpenAI
from langchain.utilities.jira import JiraAPIWrapper
from langchain.agents.agent_toolkits import JiraToolkit

from dotenv import load_dotenv
import os

from langchain_tools.slack.slack_api import SlackApiWrapper

load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')
os.environ["JIRA_USERNAME"] = os.getenv('JIRA_USERNAME')
os.environ["JIRA_API_TOKEN"] = os.getenv('JIRA_API_TOKEN')
os.environ["JIRA_INSTANCE_URL"] = os.getenv('JIRA_INSTANCE_URL')

llm = OpenAI(temperature=0)

jira = JiraAPIWrapper()
jira_toolkit = JiraToolkit.from_jira_api_wrapper(jira)

slack = SlackApiWrapper()

tools = jira_toolkit.get_tools()

agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True)

ModuleNotFoundError: No module named 'langchain_tools'

In [ ]:
agent.run("make a new issue for project key OP to remind me to make more fried rice")

In [ ]:
import requests
from requests.auth import HTTPBasicAuth
import json

base_url = os.getenv('JIRA_INSTANCE_URL')
create_url = f"{base_url}/rest/api/2/issue"

auth = HTTPBasicAuth(os.getenv('JIRA_USERNAME'), os.getenv('JIRA_API_TOKEN'))

metadata_url = f"{base_url}/rest/api/2/issue/createmeta"

headers = {
  "Accept": "application/json"
}

response = requests.request(
   "GET",
   metadata_url,
   headers=headers,
   auth=auth
)

print(json.dumps(json.loads(response.text), sort_keys=True, indent=4, separators=(",", ": ")))


In [ ]:

payload = json.dumps( {
  "fields": {
    "issuetype": {
      "name": "Task"
    },
    "priority": {
      "name": "Low"
    },
    "project": {
      "key": "OP"
    },
    "summary": "Main order flow broken",
    "description": "What do you think"
  }
} )

headers = {
  "Accept": "application/json",
  "Content-Type": "application/json"
}

response = requests.request(
   "POST",
   create_url,
   data=payload,
   headers=headers,
   auth=auth
)

print(response.status_code)
print(json.dumps(json.loads(response.text), sort_keys=True, indent=4, separators=(",", ": ")))